In [64]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(0, 'E:/GitHub/hse_workshop_regression/src')
import config as cfg

In [10]:
curr_path = '../'
train_old = pd.read_csv(curr_path + 'data/raw/train.csv')
train_old.set_index(['Id'], inplace=True)
train_old.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [21]:
for col in train[train.columns].isna().columns:
    t = (~train[col].isna()).sum()
    if t < 1460:
        print(t)

1201
91
1452
1452
1423
1423
1422
1423
1422
1459
770
1379
1379
1379
1379
1379
7
281
54


## Нужно выбрать метрику качества и обосновать ее выбор (1 балл).

Для оценки качества модели была выбрана метрика **MAE** *(Mean absolute error)*, т.к. для нас не так важно отслеживать качество предсказания для данных с сильным отклонением от среднего, и также отклонение в условные *"20"* единиц будет ровно в 2 раза хуже, чем отклонение в *"10"*, что и соотвествует ее выбору в сравнении с *RMSE*

## Нужно написать стадии для полного цикла жизни ML модели (4 баллов)

### Препроцессинг.
Описан в файле "preprocess.py". 

### Генерация признаков.

Находится в src/features/build_features.py

In [85]:
from features.build_features import feature_gen

In [71]:
train = pd.read_pickle('../data/processed/train.pkl')
target = pd.read_pickle('../data/processed/target.pkl')
test = pd.read_pickle('../data/processed/test.pkl')
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65,8450,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,2,2008,WD,Normal
2,20,RL,80,9600,Pave,0,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,0,5,2007,WD,Normal
3,60,RL,68,11250,Pave,0,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,9,2008,WD,Normal
4,70,RL,60,9550,Pave,0,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,0,2,2006,WD,Abnorml
5,60,RL,84,14260,Pave,0,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62,7917,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,8,2007,WD,Normal
1457,20,RL,85,13175,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,0,1,0,0,2,2010,WD,Normal
1458,70,RL,66,9042,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0,0,2,Shed,2500,5,2010,WD,Normal


In [ ]:
train = feature_gen(train)

### CatBoost

In [38]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

Используем **Grid-search** кросс-валидацию для подбора оптимальных гиперпараметров

In [52]:
model = CatBoostRegressor(
                        cat_features=cfg.CAT_COLS,
                        eval_metric = 'MAE',
                        random_seed = 228,
                        logging_level = 'Silent',
                        #use_best_model = True,
                        loss_function = 'MAE',
                        od_type = 'Iter',
                        one_hot_max_size = 20,
                        od_wait = 1000,
                        #rsm = 0.6,
                        bagging_temperature = 10)

grid = {'learning_rate': [0.01, 0.03, 0.05],
        'depth': [3, 6, 10],
        'l2_leaf_reg': [1, 10, 50],
        'iterations' : [1000, 5000],
        'random_strength': [1, 5]
        }

grid_search_result = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	loss: 14066.2528138	best: 14066.2528138 (0)	total: 10.5s	remaining: 18m 47s
1:	loss: 13335.6636382	best: 13335.6636382 (1)	total: 21s	remaining: 18m 34s
2:	loss: 12792.0037281	best: 12792.0037281 (2)	total: 31.9s	remaining: 18m 35s
3:	loss: 14044.1114340	best: 12792.0037281 (2)	total: 42.3s	remaining: 18m 20s
4:	loss: 13542.6253781	best: 12792.0037281 (2)	total: 53.1s	remaining: 18m 13s
5:	loss: 13203.5034114	best: 12792.0037281 (2)	total: 1m 4s	remaining: 18m 22s
6:	loss: 14450.0595902	best: 12792.0037281 (2)	total: 1m 15s	remaining: 18m 4s
7:	loss: 13238.7137520	best: 12792.0037281 (2)	total: 1m 24s	remaining: 17m 40s
8:	loss: 13185.6958067	best: 12792.0037281 (2)	total: 1m 34s	remaining: 17m 22s
9:	loss: 12734.7747622	best: 12734.7747622 (9)	total: 2m 27s	remaining: 24m 4s
10:	loss: 13235.7427448	best: 12734.7747622 (9)	total: 2m 59s	remaining: 26m 19s
11:	loss: 12792.0037281	best: 12734.7747622 (9)	total: 3m 18s	remaining: 26m 26s
12:	loss: 12931.4760504	best: 12734.7747622 (9)	

Спустя 2 часа выполнения получаем наиболее оптимальные параметры для **CatBoost**: 
- random_strength = 1
- depth = 6
- learning_rate = 0.03
- l2_leaf_reg = 50
- iterations = 5000

In [ ]:
test_pred = rf.predict(test)
pred_df=pd.DataFrame(test_pred)
sub_df=pd.read_csv('../data/raw/sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred_df], axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('../data/raw/sample_submission_rf.csv',index=False)

Теперь попробуем использовать регрессор **RandomForest**

In [72]:
def one_hot_encode(df: pd.DataFrame, ohe: OneHotEncoder) -> pd.DataFrame: 
    # df.info()
    ohe_df = pd.DataFrame(ohe.transform(df[cat_cols].astype(str)), index=df.index)
    ohe_df = ohe_df.astype(np.int8)
    df = df.merge(ohe_df, left_index=True, right_index=True)
    df = df.drop(cat_cols, axis=1)
    return df

In [73]:
def normalize_real_cols(df: pd.DataFrame, scaler: MinMaxScaler) -> pd.DataFrame:
    x_scaled = scaler.transform(df[real_cols])
    
    scaled_cols = pd.DataFrame(x_scaled, index=df.index)
    # print(scaled_cols.head())
    df = df.merge(scaled_cols, left_index=True, right_index=True)
    # print(df.head())
    df = df.drop(real_cols, axis=1)
    return df

In [74]:
int_cols = train.select_dtypes(np.int32).columns
cat_cols = train.select_dtypes('category').columns
train_tmp = train.copy()
train_tmp[int_cols] = train[int_cols].astype(np.float32)
real_cols = train_tmp.select_dtypes(np.float32).columns

In [75]:
X_train, X_val, y_train, y_val = train_test_split(train_tmp, target, train_size=0.8, random_state=42)

In [76]:
rf_train_cols = X_train.columns

In [77]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(X_train[cat_cols].astype(str))
scaler = MinMaxScaler().fit(X_train[real_cols])

In [78]:
X_train = one_hot_encode(X_train, ohe)
X_val = one_hot_encode(X_val, ohe)
test = one_hot_encode(test[rf_train_cols], ohe)
X_train = normalize_real_cols(X_train, scaler)
X_val = normalize_real_cols(X_val, scaler)
test = normalize_real_cols(test, scaler)

In [79]:
X_train.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,42_y,43_y,44_y,45_y,46_y,47_y,48_y,49_y,50_y,51_y
Id,,,,,,,,,,,,,,,,,,,,,
255,0,0,0,0,1,0,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.454545,1.00
1067,0,0,0,0,0,0,0,0,0,1,...,0.073126,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.363636,0.75
639,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.297101,0.0,0.0,0.0,0.0,0.5,0.0,0.363636,0.50
800,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.478261,0.0,0.0,0.0,0.0,0.5,0.0,0.454545,0.25
381,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.438406,0.0,0.0,0.0,0.0,0.0,0.0,0.363636,1.00


In [80]:
from sklearn.ensemble import RandomForestRegressor

In [81]:
rf = RandomForestRegressor(max_depth=5, random_state=228)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(mean_absolute_error(y_val, y_pred))

20704.762767935936


В итоге CatBoost оказался лучше

In [83]:
test_pred = rf.predict(test)
pred_df=pd.DataFrame(test_pred)
sub_df=pd.read_csv('../data/raw/sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred_df], axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('../data/raw/sample_submission_rf.csv',index=False)